In [1]:
from utils import generate_samples_linear_reg
import numpy as np

In [5]:
samples = 10
features = 2
lower_bound = 0
upper_bound = 10
intercept = 5
coeffecients = [1,3]

X, y = generate_samples_linear_reg(
    samples,
    features,
    lower_bound,
    upper_bound,
    intercept,
    coeffecients,
)

print("Shape of samples is: ",X.shape)
print("Samples are: ", X)

print("\n Shape of labels is: ",y.shape)
print("Labels are: ", y)



Shape of samples is:  (10, 2)
Samples are:  [[7.09450541 0.0885049 ]
 [2.4260621  5.13349214]
 [7.6530514  9.61398845]
 [5.11898774 8.70824329]
 [9.81298224 8.85746227]
 [8.28453505 7.64397124]
 [0.77509095 5.98724072]
 [3.72212579 4.33538306]
 [4.99308537 2.63981972]
 [3.26831771 3.20579222]]

Shape of labels is:  (10,)
Labels are:  [12.3600201  22.82653851 41.49501674 36.2437176  41.38536904 36.21644877
 23.73681311 21.72827496 17.91254453 17.88569437]


array([[ 3.94474103,  7.66187881],
       [10.5367387 , 16.57202808],
       [ 2.89760557, 15.61730904]])

In [14]:
a=np.empty((2,3))
a
a[0][0]=8
a

array([[8.00000000e+000, 3.11108246e+231, 1.42763657e-071],
       [1.85455425e-051, 1.18914750e-075, 4.17203572e-309]])

In [36]:
# np.array(([1,2,3],[3,4,5]))*np.array(([3],[4],[5]))

np.array(([1,2,3],[3,4,5]))*(np.array((3,4,5)))

# np.multiply(np.array(([1,2,3],[3,4,5]))*np.array(([3],[4],[5])))

array([[ 3,  8, 15],
       [ 9, 16, 25]])

In [38]:
np.dot(np.array(([1,2,3],[3,4,5])),(np.array((3,4,5))))

array([26, 50])

In [32]:
np.array((3,4,5)).T

array([3, 4, 5])